# 01_기본 기능

## 1. OpenAI 라이브러리 설치 확인

In [1]:
!pip install openai python-dotenv

## 2. OpenAI 정보 읽기

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv(filename='././.env')
load_dotenv(dotenv_path=dotenv_path)

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION","").strip()

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

model=os.getenv('DEPLOYMENT_NAME')

AssertionError: ERROR: Azure OpenAI Key is missing

## 3. LLM 특징 - 동일 Prompt, 다양한 Completion

### 동일 Prompt

In [ ]:
text_prompt = "Should oxford commas always be used?"

### 시도 1

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response['choices'][0]['message']['content']

### 시도 2

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response['choices'][0]['message']['content']

## 4. Openai 기본 파라미터

### temperature 

In [ ]:
def call_openai(num_times, prompt, temperature):
    for i in range(num_times):
        response = openai.ChatCompletion.create(
            engine=model,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": prompt}],
                max_tokens=60,
                temperature = temperature
        )
        print(response['choices'][0]['message']['content'])

In [ ]:
call_openai(10, 'The best pet is a ', temperature = 1)

In [ ]:
call_openai(10, 'The best pet is a ', temperature = 0)

### top_p

In [ ]:
def call_openai(num_times, prompt, top_p):
    for i in range(num_times):
        response = openai.ChatCompletion.create(
            engine=model,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": prompt}],
                max_tokens=60,
                top_p = top_p
        )
        print(response['choices'][0]['message']['content'])

In [ ]:
call_openai(10, 'The best pet is a ', temperature = 1)

In [ ]:
call_openai(10, 'The best pet is a ', temperature = 0)

### n

In [ ]:
response = openai.ChatCompletion.create(
            engine=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "The best pet is a "}],
                max_tokens=60,
                n=2
        )

In [ ]:
for c in response['choices']:
    print(c['message']['content'])

## 5. 단순 ChatCompletion JSON 구조 이해

In [ ]:
text_prompt = "Should oxford commas always be used?"

In [ ]:
openai.ChatCompletion.create(
    engine=model,
    max_tokens=60,
    messages=[
        {"role": "user", "content": text_prompt},
    ]
)

## 6. 기본 Prompt 연습

### 연습 1 - 답변 조정

In [ ]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response['choices'][0]['message']['content']

### 연습 2 - 감정 분석

In [ ]:
prompt = """Decide whether the following customer feedback is positive or negative.

Q: I was disappointed with the quality of the product. It was very cheaply made and did not meet my expectations at all.
Q: I was happy with this product, it is well made and great quality for the price.
"""

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

print(response['choices'][0]['message']['content'])

### 연습 3 - 정보 추출

In [ ]:
prompt = """List all PII data from following statement:
John Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Microsft. He has a wife named Jane Doe and they have two children
"""

In [ ]:
response = openai.ChatCompletion.create(
  engine=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

print(response['choices'][0]['message']['content'])

### 연습 4 - 쉽게 문장 요약하기

#### 'tl;dr:' 추가로 간단히 문장 요약하기  
TL;DR, "too long; didn't read"(너무 길어서 읽지 않았다)는 뜻으로, Prompt 문장 마지막에 입력하면 문장을 요약해 줌

In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response['choices'][0]['message']['content']

### 연습 5 - 문장 카테고리 분류

#### 카테고리 리스트 제공을 통한 문장 분류  

In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response['choices'][0]['message']['content']

### 연습 6 - 새로운 항목 생성

#### 사례 제공을 통한 새로운 항목 생성

In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."
print(prompt)

In [ ]:
response = openai.ChatCompletion.create(
  engine=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},]

response['choices'][0]['message']['content']